In [ ]:
%%capture
!pip install datasets
# !pip install transformers
# !pip install torchaudio

from datasets import load_dataset, load_metric, Audio
import pandas as pd
import pickle

from google.colab import drive
drive.mount('/content/drive')
import matplotlib.pyplot as plt


In [ ]:
import torch
import torchaudio
import os

In [ ]:
languages = [
  'ca',
  'de',
  'en',
  'fr',
  'zh-CN',
  'es',
  'eu',
  ]



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
new_dict = {}

for lan in languages:
  y = []
  x = []
  dataset = load_dataset("common_voice", lan,split="train", streaming=True)
  dataset = dataset.cast_column("audio", Audio(sampling_rate=16_000))
  dataset_iter = iter(dataset)

  i = 0
  counter = 0
  data_size = 3000
  axe_list = []

  while True:
    counter += 1
    try:
      sample = next(dataset_iter)

    except:
        print("~!!!!!!!!")
        

  
    y.append(lan)
    x.append(sample['audio']['array'])

    if i %500 == 0:
      print(lan)
    
    i += 1
    if i == 2000:
      print(str(lan)+" is good")
      print(axe_list)
      new_dict[lan] = axe_list
      break


  pkl_out_x = open("/content/drive/MyDrive/1_new/lbw/"+str(lan)+ "_x.pickle", "wb")
  pickle.dump(x, pkl_out_x)
  pkl_out_x.close()

  pkl_out_y = open("/content/drive/MyDrive/1_new/lbw/"+str(lan)+ "_y.pickle", "wb")
  pickle.dump(y, pkl_out_y)
  pkl_out_y.close()

   
  

ca
ca
ca
ca
ca is good
[]
de
de
de
de
de is good
[]
en
en
en
en
en is good
[]
fr
fr
fr
fr
fr is good
[]
zh-CN
zh-CN
zh-CN
zh-CN
zh-CN is good
[]
es
es
es
es
es is good
[]
eu
eu
eu
eu
eu is good
[]


In [ ]:
os.chdir("/content/drive/MyDrive/1_new/lbw")
filepash = os.listdir()
filepash

['ca_x.pickle',
 'ca_y.pickle',
 'de_x.pickle',
 'de_y.pickle',
 'en_x.pickle',
 'en_y.pickle',
 'fr_x.pickle',
 'fr_y.pickle',
 'zh-CN_x.pickle',
 'zh-CN_y.pickle',
 'es_x.pickle',
 'es_y.pickle',
 'eu_x.pickle',
 'eu_y.pickle']

In [ ]:
y = []
x = []
for z in filepash:
  if "_y" in z:
    pickle_in_y = open("/content/drive/MyDrive/1_new/lbw/"+ z,"rb")
    y_load = pickle.load(pickle_in_y)
    y.extend(y_load)
  else:
    pickle_in_x = open("/content/drive/MyDrive/1_new/lbw/" + z,"rb")
    x_load = pickle.load(pickle_in_x)
    x.extend(x_load)

In [ ]:
data = []
for i in range(len(x)):
  if len(x[i]) >= 48044:
    data.append((x[i][44:], y[i]))



In [ ]:
print(len(data))
print(data[1])

11645
(array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
        5.4458658e-05, -1.5439701e-04,  3.4494521e-04], dtype=float32), 'ca')


In [ ]:
import random
random.shuffle(data)

x_train_split = []
y_train_split = []
x_test_split = []
y_test_split = []

x_train_no_split, y_train_no_split = zip(*data[:int(len(data)*0.8)])
x_test_no_split, y_test_no_split = zip(*data[int(len(data)*0.8):])
for i in range(len(x_train_no_split)):
  temp_x = x_train_no_split[i]
  temp_y = y_train_no_split[i]
  splits = int(len(temp_x)/48000)
  for j in range(splits):
    x_train_split.append(temp_x[j*48000:(j+1)*48000])
    y_train_split.append(temp_y)

for i in range(len(x_test_no_split)):
  temp_x = x_test_no_split[i]
  temp_y = y_test_no_split[i]
  splits = int(len(temp_x)/48000)
  for j in range(splits):
    x_test_split.append(temp_x[j*48000:(j+1)*48000])
    y_test_split.append(temp_y)


In [ ]:
print(len(x_test_split[0]))

48000


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
x_train_tensor = [torch.tensor([t]).to(device) for t in x_train_split]
x_test_tensor = [torch.tensor([t]).to(device) for t in x_test_split]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  """Entry point for launching an IPython kernel.


In [ ]:
bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
model = bundle.get_model().to(device)
x_train_tensor[0].shape

Downloading: "https://download.pytorch.org/torchaudio/models/wav2vec2_fairseq_base_ls960_asr_ls960.pth" to /root/.cache/torch/hub/checkpoints/wav2vec2_fairseq_base_ls960_asr_ls960.pth


  0%|          | 0.00/360M [00:00<?, ?B/s]

torch.Size([1, 48000])

In [ ]:
x_train_w2v = []
x_test_w2v = []
y_train_w2v = []
y_test_w2v = []


In [ ]:
counter = -1
while counter != len(x_train_tensor):
  for w in range(1000):
    counter +=1 
    with torch.inference_mode():
      try:
        emission, _ = model(x_train_tensor[counter])
        x_train_w2v.append(emission.to("cpu"))
        if counter % 100 == 0:
          print(counter)
      except:
        print("break!!!")
        break
# pkl_out_x = open("/content/drive/MyDrive/1_new/law/all_x_test.pickle", "wb")
# pickle.dump(x_test_w2v, pkl_out_x)
# pkl_out_x.close()
# x_test_w2v = []
  
print("save")



0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
break!!!
save


In [ ]:
pkl_out_x = open("/content/drive/MyDrive/1_new/law/all_y_train.pickle", "wb")
pickle.dump(y_train_split, pkl_out_x)
pkl_out_x.close()

In [ ]:
pkl_out_x = open("/content/drive/MyDrive/1_new/law/all_x_train.pickle", "wb")
pickle.dump(x_train_w2v, pkl_out_x)
pkl_out_x.close()



In [ ]:
counter = -1
while counter != len(x_test_tensor):
  for w in range(1000):
    counter +=1 
    with torch.inference_mode():
      try:
        emission, _ = model(x_test_tensor[counter])
        x_test_w2v.append(emission.to("cpu"))
        if counter % 100 == 0:
          print(counter)
      except:
        print("break!!!")
        break
# pkl_out_x = open("/content/drive/MyDrive/1111/lan_pkl/all_x_test.pickle", "wb")
# pickle.dump(x_test_w2v, pkl_out_x)
# pkl_out_x.close()
# x_test_w2v = []
  
print("save")


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
break!!!
save


In [ ]:
pkl_out_x = open("/content/drive/MyDrive/1_new/law/all_y_test.pickle", "wb")
pickle.dump(y_test_split, pkl_out_x)
pkl_out_x.close()

pkl_out_x = open("/content/drive/MyDrive/1_new/law/all_x_test.pickle", "wb")
pickle.dump(x_test_w2v, pkl_out_x)
pkl_out_x.close()

